# **Extracting Information from Legal Documents Using RAG**

## **Objective**

The main objective of this assignment is to process and analyse a collection text files containing legal agreements (e.g., NDAs) to prepare them for implementing a **Retrieval-Augmented Generation (RAG)** system. This involves:

* Understand the Cleaned Data : Gain a comprehensive understanding of the structure, content, and context of the cleaned dataset.
* Perform Exploratory Analysis : Conduct bivariate and multivariate analyses to uncover relationships and trends within the cleaned data.
* Create Visualisations : Develop meaningful visualisations to support the analysis and make findings interpretable.
* Derive Insights and Conclusions : Extract valuable insights from the cleaned data and provide clear, actionable conclusions.
* Document the Process : Provide a detailed description of the data, its attributes, and the steps taken during the analysis for reproducibility and clarity.

The ultimate goal is to transform the raw text data into a clean, structured, and analysable format that can be effectively used to build and train a RAG system for tasks like information retrieval, question-answering, and knowledge extraction related to legal agreements.

### **Business Value**  


The project aims to leverage RAG to enhance legal document processing for businesses, law firms, and regulatory bodies. The key business objectives include:

* Faster Legal Research: <br> Reduce the time lawyers and compliance officers spend searching for relevant case laws, precedents, statutes, or contract clauses.
* Improved Contract Analysis: <br> Automatically extract key terms, obligations, and risks from lengthy contracts.
* Regulatory Compliance Monitoring: <br> Help businesses stay updated with legal and regulatory changes by retrieving relevant legal updates.
* Enhanced Decision-Making: <br> Provide accurate and context-aware legal insights to assist in risk assessment and legal strategy.


**Use Cases**
* Legal Chatbots
* Contract Review Automation
* Tracking Regulatory Changes and Compliance Monitoring
* Case Law Analysis of past judgments
* Due Diligence & Risk Assessment

## **1. Data Loading, Preparation and Analysis** <font color=red> [20 marks] </font><br>

### **1.1 Data Understanding**

The dataset contains legal documents and contracts collected from various sources. The documents are present as text files (`.txt`) in the *corpus* folder.

There are four types of documents in the *courpus* folder, divided into four subfolders.
- `contractnli`: contains various non-disclosure and confidentiality agreements
- `cuad`: contains contracts with annotated legal clauses
- `maud`: contains various merger/acquisition contracts and agreements
- `privacy_qa`: a question-answering dataset containing privacy policies

The dataset also contains evaluation files in JSON format in the *benchmark* folder. The files contain the questions and their answers, along with sources. For each of the above four folders, there is a `json` file: `contractnli.json`, `cuad.json`, `maud.json` `privacy_qa.json`. The file structure is as follows:

```
{
    "tests": [
        {
            "query": <question1>,
            "snippets": [{
                    "file_path": <source_file1>,
                    "span": [ begin_position, end_position ],
                    "answer": <relevant answer to the question 1>
                },
                {
                    "file_path": <source_file2>,
                    "span": [ begin_position, end_position ],
                    "answer": <relevant answer to the question 2>
                }, ....
            ]
        },
        {
            "query": <question2>,
            "snippets": [{<answer context for que 2>}]
        },
        ... <more queries>
    ]
}
```

### **1.2 Load and Preprocess the data** <font color=red> [5 marks] </font><br>

#### Loading libraries

In [12]:
# Install necessary libraries
!pip install -q langchain-community 
!pip install -q langchain-groq     
!pip install -q faiss-cpu         
!pip install -q sentence-transformers
!pip install -q langchain-huggingface
!pip install -q tf-keras
!pip install -q datasets
!pip install -q ragas
!pip install -q rouge_score

In [13]:
# Import essential libraries
import os
import re
import glob
from collections import Counter
import random

# For data loading and preprocessing
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# For EDA and similarity analysis
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Download NLTK data (if not already downloaded)
try:
    stopwords.words('english')
except LookupError:
    nltk.download('stopwords')

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

# *** NEW ADDITION TO ADDRESS YOUR SPECIFIC ERROR ***
try:
    nltk.data.find('tokenizers/punkt_tab') # Explicitly check for punkt_tab
except LookupError:
    print("Downloading 'punkt_tab' resource...")
    nltk.download('punkt_tab') # Download 'punkt_tab' if missing
# ***************************************************

print("Libraries imported successfully.")

Libraries imported successfully.


#### **1.2.1** <font color=red> [3 marks] </font>
Load all `.txt` files from the folders.

You can utilise document loaders from the options provided by the LangChain community.

Optionally, you can also read the files manually, while ensuring proper handling of encoding issues (e.g., utf-8, latin1). In such case, also store the file content along with metadata (e.g., file name, directory path) for traceability.

In [14]:
# Load the files as documents

# Define the path to the main corpus directory
corpus_path = './rag_legal/corpus/'

# Use DirectoryLoader to load all .txt files from the subdirectories
# glob="**/[!.]*" ensures we get files from subdirectories
# loader_cls=TextLoader specifies how to load each file, with encoding handled
loader = DirectoryLoader(
    corpus_path,
    glob="**/*.txt",
    loader_cls=TextLoader,
    loader_kwargs={'autodetect_encoding': True},
    show_progress=True,
    use_multithreading=True
)

documents = loader.load()

# Log the number of documents loaded
print(f"Successfully loaded {len(documents)} documents.")

# Optional: Display a sample document to verify
if documents:
    print("\n--- Sample Document ---")
    print(f"Source: {documents[0].metadata['source']}")
    print(f"Content Preview: {documents[0].page_content[:500]}")
    print("-----------------------")

100%|██████████| 698/698 [00:09<00:00, 77.22it/s] 

Successfully loaded 698 documents.

--- Sample Document ---
Source: rag_legal\corpus\contractnli\12032018_NDA_The_20Munt_EN.txt
Content Preview: Confidentiality Agreement
This Confidentiality Agreement (the “Agreement”) has been prepared on behalf of bpost SA/NV, a limited liability company of public law, and the City of Brussels (the “Sellers”) with regard to the divestment of their respective surface areas (the “Sale”) in the Munt Center/Centre Monnaie (the “Munt”).
In consideration of the Sellers agreeing to disclose confidential information (the “Confidential Information”) to the Candidate (“Disclosee”), the latter hereby undertakes 
-----------------------


#### **1.2.2** <font color=red> [2 marks] </font>
Preprocess the text data to remove noise and prepare it for analysis.

Remove special characters, extra whitespace, and irrelevant content such as email and telephone contact info.
Normalise text (e.g., convert to lowercase, remove stop words).
Handle missing or corrupted data by logging errors and skipping problematic files.

In [15]:
# Clean and preprocess the data
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    """
    Cleans and preprocesses a given text by:
    1. Converting to lowercase.
    2. Removing special characters and numbers.
    3. Removing extra whitespace.
    4. Removing email addresses and phone numbers.
    5. Tokenizing and removing stopwords.
    """
    if not isinstance(text, str):
        return ""

    # Convert to lowercase
    text = text.lower()

    # Remove email addresses
    text = re.sub(r'\S+@\S+', '', text)
    # Remove telephone numbers (basic patterns)
    text = re.sub(r'\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}', '', text)

    # Remove special characters and numbers, keeping only letters
    text = re.sub(r'[^a-z\s]', '', text)

    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    # Tokenize and remove stopwords
    word_tokens = word_tokenize(text)
    filtered_words = [word for word in word_tokens if word not in stop_words]

    return " ".join(filtered_words)

# Assuming 'documents' is defined elsewhere in your script.
# For demonstration purposes, let's create a dummy list if it's not present:
class Document:
    def __init__(self, page_content, source):
        self.page_content = page_content
        self.metadata = {'source': source}

# Example usage (replace with your actual 'documents' loading)
if 'documents' not in locals(): # Only create dummy if 'documents' not already defined
    documents = [
        Document("This is a sample document with some text. My email is test@example.com and my phone is 123-456-7890. It has some numbers like 123.", "source_a"),
        Document("Another document here. Call me at (987) 654 3210. This is great!", "source_b"),
        Document("Third document with different content. No contact info here.", "source_c")
    ]

# Apply preprocessing to all documents
# Note: This step can take a few moments
for doc in documents:
    doc.page_content = preprocess_text(doc.page_content)

print("Preprocessing complete for all documents.")
# Display a sample of preprocessed content
if documents:
    print("\n--- Sample Preprocessed Document ---")
    print(f"Source: {documents[0].metadata['source']}")
    print(f"Content Preview: {documents[0].page_content[:500]}")
    print("----------------------------------")

Preprocessing complete for all documents.

--- Sample Preprocessed Document ---
Source: rag_legal\corpus\contractnli\12032018_NDA_The_20Munt_EN.txt
Content Preview: confidentiality agreement confidentiality agreement agreement prepared behalf bpost sanv limited liability company public law city brussels sellers regard divestment respective surface areas sale munt centercentre monnaie munt consideration sellers agreeing disclose confidential information confidential information candidate disclosee latter hereby undertakes sellers disclosee keep confidential confidential information received time disclosee procure prior disclosure person including professiona
----------------------------------


### **1.3 Exploratory Data Analysis** <font color=red> [10 marks] </font><br>

#### **1.3.1** <font color=red> [1 marks] </font>
Calculate the average, maximum and minimum document length.

In [16]:
# Calculate the average, maximum and minimum document length.
doc_lengths = [len(word_tokenize(doc.page_content)) for doc in documents]

# Filter out zero-length documents if any exist after preprocessing
doc_lengths = [length for length in doc_lengths if length > 0]

if doc_lengths:
    average_length = sum(doc_lengths) / len(doc_lengths)
    max_length = max(doc_lengths)
    min_length = min(doc_lengths)

    print(f"Document Length Analysis (in words):")
    print(f"Average Document Length: {average_length:.2f}")
    print(f"Maximum Document Length: {max_length}")
    print(f"Minimum Document Length: {min_length}")
else:
    print("No documents with content were found to analyze.")

Document Length Analysis (in words):
Average Document Length: 8620.66
Maximum Document Length: 82545
Minimum Document Length: 132


#### **1.3.2** <font color=red> [4 marks] </font>
Analyse the frequency of occurrence of words and find the most and least occurring words.

Find the 20 most common and least common words in the text. Ignore stop words such as articles and prepositions.

In [17]:
# Find frequency of occurence of words

# Concatenate all document contents into a single text block
full_text = " ".join([doc.page_content for doc in documents])

# Tokenize the full text
all_words = word_tokenize(full_text)

# Calculate word frequencies
word_counts = Counter(all_words)

# Find the 20 most common and 20 least common words
most_common_words = word_counts.most_common(20)
least_common_words = word_counts.most_common()[-20:]

print("Top 20 Most Common Words:")
for word, count in most_common_words:
    print(f"- {word}: {count}")

print("\nTop 20 Least Common Words (often unique terms or typos):")
for word, count in reversed(least_common_words): # reversed to show least frequent first
    print(f"- {word}: {count}")

Top 20 Most Common Words:
- company: 148175
- shall: 107998
- agreement: 104577
- section: 75368
- parent: 58010
- party: 49664
- date: 39300
- time: 35256
- material: 34211
- merger: 33848
- subsidiaries: 33318
- b: 31719
- applicable: 31370
- including: 29399
- respect: 28848
- may: 28114
- stock: 26654
- information: 25687
- parties: 24614
- business: 23531

Top 20 Least Common Words (often unique terms or typos):
- newer: 1
- peoplefuncom: 1
- nonmarketing: 1
- httpwwwyouronlinechoiceseu: 1
- httpwwwaboutadsinfochoices: 1
- checkins: 1
- httpsvunglecomprivacypolicy: 1
- vungle: 1
- httpswwwtapresearchcomuserprivacy: 1
- tapresearch: 1
- httpsdevtapjoycomfaqtapjoyprivacypolicy: 1
- tapjoy: 1
- httpswwwstartappcompolicyprivacypolicy: 1
- startappcom: 1
- httpaboutsoomlaenduserprivacypolicy: 1
- soomla: 1
- httpswwwsmaatocomprivacy: 1
- smaato: 1
- httpspinsightmediacomprivacy: 1
- pinsight: 1


#### **1.3.3** <font color=red> [4 marks] </font>
Analyse the similarity of different documents to each other based on TF-IDF vectors.

Transform some documents to TF-IDF vectors and calculate their similarity matrix using a suitable distance function. If contracts contain duplicate or highly similar clauses, similarity calculation can help detect them.

Identify for the first 10 documents and then for 10 random documents. What do you observe?

In [18]:
# Transform the page contents of documents
# Extract page content for the first 10 documents
docs_content = [doc.page_content for doc in documents[:10]]
doc_sources = [doc.metadata['source'] for doc in documents[:10]]

# Initialise TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the documents to TF-IDF vectors
tfidf_matrix = tfidf_vectorizer.fit_transform(docs_content)

# Compute similarity scores
# Calculate cosine similarity between all pairs of documents
similarity_matrix = cosine_similarity(tfidf_matrix)

print("--- TF-IDF Cosine Similarity Matrix (First 10 Documents) ---")
# Print the matrix, rounding for readability
print(np.round(similarity_matrix, 2))
print("\nObservation:")
print("The similarity matrix shows how related each document is to every other document in the set.")
print("A value of 1.0 on the diagonal is expected, as a document is perfectly similar to itself.")
print("Off-diagonal values close to 1 suggest high similarity (e.g., similar contracts), while values close to 0 suggest dissimilarity.")


--- TF-IDF Cosine Similarity Matrix (First 10 Documents) ---
[[1.   0.27 0.2  0.27 0.22 0.23 0.23 0.25 0.25 0.2 ]
 [0.27 1.   0.48 0.61 0.52 0.57 0.59 0.71 0.45 0.48]
 [0.2  0.48 1.   0.64 0.57 0.61 0.61 0.59 0.3  0.56]
 [0.27 0.61 0.64 1.   0.71 0.73 0.74 0.76 0.42 0.66]
 [0.22 0.52 0.57 0.71 1.   0.61 0.62 0.62 0.33 0.58]
 [0.23 0.57 0.61 0.73 0.61 1.   0.72 0.63 0.37 0.64]
 [0.23 0.59 0.61 0.74 0.62 0.72 1.   0.66 0.39 0.63]
 [0.25 0.71 0.59 0.76 0.62 0.63 0.66 1.   0.36 0.54]
 [0.25 0.45 0.3  0.42 0.33 0.37 0.39 0.36 1.   0.32]
 [0.2  0.48 0.56 0.66 0.58 0.64 0.63 0.54 0.32 1.  ]]

Observation:
The similarity matrix shows how related each document is to every other document in the set.
A value of 1.0 on the diagonal is expected, as a document is perfectly similar to itself.
Off-diagonal values close to 1 suggest high similarity (e.g., similar contracts), while values close to 0 suggest dissimilarity.


In [19]:
# create a list of 10 random integers
# Ensure we don't pick an index that is out of bounds
num_docs = len(documents)
random_indices = random.sample(range(num_docs), 10)

print("Generated 10 random indices for document selection:")
print(random_indices)

Generated 10 random indices for document selection:
[58, 685, 438, 182, 513, 615, 119, 3, 549, 335]


In [20]:
# Compute similarity scores for 10 random documents
# Extract content and sources for the randomly selected documents
random_docs_content = [documents[i].page_content for i in random_indices]
random_doc_sources = [documents[i].metadata['source'] for i in random_indices]

# Transform the documents to TF-IDF vectors
# We can use the same vectorizer instance
random_tfidf_matrix = tfidf_vectorizer.fit_transform(random_docs_content)

# Calculate cosine similarity
random_similarity_matrix = cosine_similarity(random_tfidf_matrix)

print(f"--- TF-IDF Cosine Similarity Matrix ({len(random_indices)} Random Documents) ---")
print("Selected document sources:")
for i, source in enumerate(random_doc_sources):
    print(f"Doc {i}: {os.path.basename(source)}")

print("\nSimilarity Matrix:")
print(np.round(random_similarity_matrix, 2))

print("\nObservation:")
print("By analyzing a random set, we get a broader sense of the diversity within the entire corpus.")
print("If the random documents come from different subfolders (e.g., one from 'maud' and one from 'privacy_qa'), their similarity scores are likely to be lower than if they were from the same category, reflecting their different legal purposes.")


--- TF-IDF Cosine Similarity Matrix (10 Random Documents) ---
Selected document sources:
Doc 0: NDA-SqillerBetaTesters.txt
Doc 1: Waddell _ Reed Financial, Inc._Macquarie Group Limited.txt
Doc 2: ReedsInc_20191113_10-Q_EX-10.4_11888303_EX-10.4_Development Agreement.txt
Doc 3: ChinaRealEstateInformationCorp_20090929_F-1_EX-10.32_4771615_EX-10.32_Content License Agreement.txt
Doc 4: VAXCYTE,INC_05_22_2020-EX-10.19-SUPPLY AGREEMENT.txt
Doc 5: Iconix_Brand_Group_Lancer Capital.txt
Doc 6: AMERICASSHOPPINGMALLINC_12_10_1999-EX-10.2-SITE DEVELOPMENT AND HOSTING AGREEMENT.txt
Doc 7: 5-NSK-Confidentiality-Agreement-for-Suppliers.txt
Doc 8: XinhuaSportsEntertainmentLtd_20070221_F-1_EX-99.4_645553_EX-99.4_Content License Agreement.txt
Doc 9: LegacyEducationAllianceInc_20200330_10-K_EX-10.18_12090678_EX-10.18_Development Agreement.txt

Similarity Matrix:
[[1.   0.2  0.31 0.31 0.17 0.22 0.29 0.72 0.2  0.21]
 [0.2  1.   0.31 0.25 0.17 0.82 0.3  0.21 0.15 0.15]
 [0.31 0.31 1.   0.26 0.18 0.31 0.29 0.

### **1.4 Document Creation and Chunking** <font color=red> [5 marks] </font><br>

#### **1.4.1** <font color=red> [5 marks] </font>
Perform appropriate steps to split the text into chunks.

In [21]:
# Process files and generate chunks
# Process files and generate chunks
# Initialize the text splitter
# RecursiveCharacterTextSplitter is robust for varied text structures
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # The size of each chunk in characters
    chunk_overlap=200,  # The overlap between consecutive chunks
    length_function=len,
    is_separator_regex=False,
)

# Split the documents into chunks
chunks = text_splitter.split_documents(documents)

print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

# Optional: Display a sample chunk to verify
if chunks:
    print("\n--- Sample Chunk ---")
    print(f"Source: {chunks[0].metadata['source']}")
    print(f"Content Preview: {chunks[0].page_content}")
    print("--------------------")


Split 698 documents into 62047 chunks.

--- Sample Chunk ---
Source: rag_legal\corpus\contractnli\12032018_NDA_The_20Munt_EN.txt
Content Preview: confidentiality agreement confidentiality agreement agreement prepared behalf bpost sanv limited liability company public law city brussels sellers regard divestment respective surface areas sale munt centercentre monnaie munt consideration sellers agreeing disclose confidential information confidential information candidate disclosee latter hereby undertakes sellers disclosee keep confidential confidential information received time disclosee procure prior disclosure person including professional advisor confidential information person made aware provisions agreement fact disclosee liable disclosee acknowledges confidential information far relates bpost sanv could price sensitive information ensures compliance relevant legal provisions respect including without limited provisions belgian law august supervision financial sector financial servi

## **2. Vector Database and RAG Chain Creation** <font color=red> [15 marks] </font><br>

### **2.1 Vector Embedding and Vector Database Creation** <font color=red> [7 marks] </font><br>

#### **2.1.1** <font color=red> [2 marks] </font>
Initialise an embedding function for loading the embeddings into the vector database.

Initialise a function to transform the text to vectors using OPENAI Embeddings module. You can also use this function to transform during vector DB creation itself.

In [22]:
# Fetch your GROQ API Key as an environment variable
import os

# Set your Groq API key from environment variables
# You should set this in your environment (e.g., in your shell: export GROQ_API_KEY='your_key_here')
# Or, if running in a Colab notebook, you can do:
#from google.colab import userdata
os.environ["GROQ_API_KEY"] = "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"

if "GROQ_API_KEY" not in os.environ:
    print("GROQ_API_KEY environment variable not found. Please set it to proceed.")
else:
    print("GROQ_API_KEY loaded successfully from environment variables.")

GROQ_API_KEY loaded successfully from environment variables.


In [23]:
# Initialise an embedding function
from langchain_community.embeddings import HuggingFaceEmbeddings

# Initialize a HuggingFace embedding model
# You can choose various models from Hugging Face.
# 'sentence-transformers/all-MiniLM-L6-v2' is a good general-purpose model.
# For more specific legal contexts, you might look for domain-specific models on Hugging Face.
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

print(f"Embedding model '{model_name}' initialized successfully.")

Embedding model 'sentence-transformers/all-MiniLM-L6-v2' initialized successfully.


#### **2.1.2** <font color=red> [5 marks] </font>
Load the embeddings to a vector database.

Create a directory for vector database and enter embedding data to the vector DB.

In [24]:
# Add Chunks to vector DB

from langchain_community.vectorstores import FAISS

# Assuming 'chunks' is your list of Document objects after chunking
# from the earlier preprocessing steps.

# Create a FAISS vector database from the document chunks and the embeddings
vectorstore = FAISS.from_documents(chunks, embeddings)

print(f"FAISS vector store created with {len(chunks)} chunks.")

# You can save the vector store to disk if you want to reuse it later without re-embedding
# vectorstore.save_local("faiss_legal_documents")

# To load later:
# from langchain_community.vectorstores import FAISS
# from langchain_community.embeddings import HuggingFaceEmbeddings
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
# loaded_vectorstore = FAISS.load_local("faiss_legal_documents", embeddings, allow_dangerous_deserialization=True)

FAISS vector store created with 62047 chunks.


### **2.2 Create RAG Chain** <font color=red> [8 marks] </font><br>

#### **2.2.1** <font color=red> [5 marks] </font>
Create a RAG chain.

In [25]:
# Create a RAG chain
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA

groq_llm = ChatGroq(temperature=0, model_name="llama3-8b-8192")

retriever = vectorstore.as_retriever()

qa_chain = RetrievalQA.from_chain_type(
    llm=groq_llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

print("RAG chain with Groq LLM and FAISS retriever initialized.")

RAG chain with Groq LLM and FAISS retriever initialized.


#### **2.2.2** <font color=red> [3 marks] </font>
Create a function to generate answer for asked questions.

Use the RAG chain to generate answer for a question and provide source documents

In [27]:
# Create a function for question answering
def get_answer(query: str):
    """
    Invokes the RAG chain with a query and returns the answer.
    Also returns the source documents used for the answer.
    """
    # We need to get the source documents from the retriever separately
    source_docs = retriever.invoke(query)
    answer = qa_chain.invoke(query)
    return {
        "answer": answer,
        "source_documents": source_docs
    }

# Test the function with the example question
question = "Consider the Non-Disclosure Agreement between CopAcc and ToP Mentors; Does the document indicate that the Agreement does not grant the Receiving Party any rights to the Confidential Information?"
result = get_answer(question)

print("--- Example Question & Answer ---")
print(f"Question: {question}")
print(f"\nAnswer: {result['answer']}")

# Print the sources used for the answer
print("\n--- Sources Used ---")
for i, doc in enumerate(result['source_documents']):
    print(f"Source {i+1}: {os.path.basename(doc.metadata['source'])}")
    print(f"Content: {doc.page_content[:250]}...\n")
print("--------------------------")


--- Example Question & Answer ---
Question: Consider the Non-Disclosure Agreement between CopAcc and ToP Mentors; Does the document indicate that the Agreement does not grant the Receiving Party any rights to the Confidential Information?

Answer: {'query': 'Consider the Non-Disclosure Agreement between CopAcc and ToP Mentors; Does the document indicate that the Agreement does not grant the Receiving Party any rights to the Confidential Information?', 'result': 'No, the document does not indicate that the Agreement does not grant the Receiving Party any rights to the Confidential Information. In fact, the document states that the Receiving Party may disclose Confidential Information to the extent required without liability, and that the Agreement does not grant any rights to the Confidential Information.', 'source_documents': [Document(id='09c36731-ef03-46e8-8e3a-35e562d1fd66', metadata={'source': 'rag_legal\\corpus\\contractnli\\wayne-fueling-systems-mutual-non-disclosure-agreement-fi

## **3. RAG Evaluation** <font color=red> [10 marks] </font><br>

### **3.1 Evaluation and Inference** <font color=red> [10 marks] </font><br>

#### **3.1.1** <font color=red> [2 marks] </font>
Extract all the questions and all the answers/ground truths from the benchmark files.

Create a questions set and an answers set containing all the questions and answers from the benchmark files to run evaluations.

In [28]:
# Create a question set by taking all the questions from the benchmark data
# Also create a ground truth/answer set
import json

benchmark_folder = './rag_legal/benchmarks/'
questions = []
ground_truths = []

# List all json files in the benchmark folder
json_files = [f for f in os.listdir(benchmark_folder) if f.endswith('.json')]

for file_name in json_files:
    file_path = os.path.join(benchmark_folder, file_name)
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
        for test in data.get('tests', []):
            questions.append(test['query'])
            # The ground truth is the "answer" from the first snippet
            # This is an assumption based on the data structure
            if test['snippets']:
                ground_truths.append(test['snippets'][0]['answer'])
            else:
                ground_truths.append("") # Append empty string if no answer is provided

print(f"Extracted {len(questions)} questions and {len(ground_truths)} ground truths from benchmark files.")

# Display a sample
print("\n--- Sample Question and Ground Truth ---")
print(f"Question: {questions[0]}")
print(f"Ground Truth: {ground_truths[0]}")
print("--------------------------------------")

Extracted 6889 questions and 6889 ground truths from benchmark files.

--- Sample Question and Ground Truth ---
Question: Consider the Non-Disclosure Agreement between CopAcc and ToP Mentors; Does the document indicate that the Agreement does not grant the Receiving Party any rights to the Confidential Information?
Ground Truth: Any and all proprietary rights, including but not limited to rights to and in inventions, patent rights, utility models, copyrights, trademarks and trade secrets, in and to any Confidential Information shall be and remain with the Participants respectively, and Mentor shall not have any right, license, title or interest in or to any Confidential Information, except the limited right to review, assess and help develop such Confidential Information in connection with the Copernicus Accelerator 2017.
--------------------------------------


#### **3.1.2** <font color=red> [5 marks] </font>
Create a function to evaluate the generated answers.

Evaluate the responses on *Rouge*, *Ragas* and *Bleu* scores.

In [34]:
# Function to evaluate the RAG pipeline
# Installing necessary libraries for evaluation
import pandas as pd
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import numpy as np # Ensure numpy is imported

# Assuming 'qa_chain' is your RAG chain initialized previously (from Groq LLM and FAISS)

def evaluate_rag_pipeline(questions, ground_truths, limit=None):
    """
    Evaluates the RAG pipeline using ROUGE and BLEU scores.

    Args:
        questions (list): A list of question strings.
        ground_truths (list): A list of dictionaries, where each dictionary
                              contains the ground truth answer (e.g., {'answer': '...'}),
                              or plain string ground truths.
        limit (int, optional): The number of samples to evaluate. Defaults to None (all samples).

    Returns:
        dict: A dictionary containing evaluation results, including average scores
              and samples of generated vs. ground truth answers.
    """
    if limit:
        questions_subset = questions[:limit]
        ground_truths_subset = ground_truths[:limit]
    else:
        questions_subset = questions
        ground_truths_subset = ground_truths

    generated_answers = []
    # Ensure ground_truths are processed into a list of strings
    relevant_ground_truths = []
    for gt in ground_truths_subset:
        if isinstance(gt, dict) and 'answer' in gt and isinstance(gt['answer'], str):
            relevant_ground_truths.append(gt['answer'])
        elif isinstance(gt, str): # Handle cases where ground_truth might already be a string
            relevant_ground_truths.append(gt)
        else:
            # Fallback for unexpected types, convert to string
            print(f"Warning: Unexpected type or format for ground_truth: {type(gt)} - {gt}. Converting to string.")
            relevant_ground_truths.append(str(gt))


    print(f"Generating answers for {len(questions_subset)} questions...")
    for i, question in enumerate(questions_subset):
        try:
            # qa_chain.invoke returns a dictionary, we need to extract the 'result' key
            response = qa_chain.invoke({"query": question})
            # Ensure the generated answer is a string
            generated_answer = str(response.get('result', ''))
            generated_answers.append(generated_answer)
            if (i + 1) % 10 == 0 or (i + 1) == len(questions_subset):
                print(f"Processed {i+1}/{len(questions_subset)} questions.")
        except Exception as e:
            print(f"Error generating answer for question '{question}': {e}")
            generated_answers.append("") # Append empty string on error


    # Initialize ROUGE scorer
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}
    bleu_scores = []
    smoothing_function = SmoothingFunction().method1 # For BLEU score calculation

    print("Calculating ROUGE and BLEU scores...")
    for j, (gen, ref) in enumerate(zip(generated_answers, relevant_ground_truths)):
        # Ensure both gen and ref are non-empty strings before scoring
        gen_str = str(gen).strip()
        ref_str = str(ref).strip()

        if not gen_str:
            # print(f"Warning: Generated answer at index {j} is empty. Skipping ROUGE/BLEU for this sample.")
            rouge_scores['rouge1'].append(0.0)
            rouge_scores['rouge2'].append(0.0)
            rouge_scores['rougeL'].append(0.0)
            bleu_scores.append(0.0)
            continue # Skip to next iteration

        if not ref_str:
            # print(f"Warning: Ground truth at index {j} is empty. Skipping ROUGE/BLEU for this sample.")
            rouge_scores['rouge1'].append(0.0)
            rouge_scores['rouge2'].append(0.0)
            rouge_scores['rougeL'].append(0.0)
            bleu_scores.append(0.0)
            continue # Skip to next iteration

        try:
            # ROUGE Score
            scores = scorer.score(ref_str, gen_str) # Pass guaranteed strings
            rouge_scores['rouge1'].append(scores['rouge1'].fmeasure)
            rouge_scores['rouge2'].append(scores['rouge2'].fmeasure)
            rouge_scores['rougeL'].append(scores['rougeL'].fmeasure)

            # BLEU Score (requires tokenized sentences)
            # Reference should be a list of lists of words (e.g., [['word1', 'word2']])
            # Hypothesis should be a list of words
            reference_tokens = [word_tokenize(ref_str.lower())]
            hypothesis_tokens = word_tokenize(gen_str.lower())
            if len(hypothesis_tokens) >= 4:
                bleu_scores.append(sentence_bleu(reference_tokens, hypothesis_tokens, smoothing_function=smoothing_function))
            else:
                bleu_scores.append(0.0) # Assign 0 if hypothesis is too short for BLEU
        except Exception as e:
            print(f"Error calculating scores for sample {j}: {e}")
            print(f"Ref type: {type(ref_str)}, content: '{ref_str[:100]}...'")
            print(f"Gen type: {type(gen_str)}, content: '{gen_str[:100]}...'")
            rouge_scores['rouge1'].append(0.0)
            rouge_scores['rouge2'].append(0.0)
            rouge_scores['rougeL'].append(0.0)
            bleu_scores.append(0.0)


    avg_rouge1 = np.mean(rouge_scores['rouge1']) if rouge_scores['rouge1'] else 0
    avg_rouge2 = np.mean(rouge_scores['rouge2']) if rouge_scores['rouge2'] else 0
    avg_rougeL = np.mean(rouge_scores['rougeL']) if rouge_scores['rougeL'] else 0
    avg_bleu = np.mean(bleu_scores) if bleu_scores else 0

    return {
        'avg_rouge1': avg_rouge1,
        'avg_rouge2': avg_rouge2,
        'avg_rougeL': avg_rougeL,
        'avg_bleu': avg_bleu,
        'questions': questions_subset,
        'ground_truths': relevant_ground_truths,
        'generated_answers': generated_answers
    }

print("evaluate_rag_pipeline function (refined) defined.")
print("Evaluation function is ready.")

evaluate_rag_pipeline function (refined) defined.
Evaluation function is ready.


#### **3.1.3** <font color=red> [3 marks] </font>
Draw inferences by evaluating answers to all questions.

To save time and computing power, you can just run the evaluation on first 100 questions.

In [37]:
# Evaluate the RAG pipeline
# Set the evaluation limit to 100 as requested.
evaluation_limit = 100
evaluation_results = evaluate_rag_pipeline(questions, ground_truths, limit=evaluation_limit)

print("\n--- RAG Pipeline Evaluation Results ---")
print(f"Evaluated on {evaluation_limit} samples.")
print(f"Average ROUGE-1 F-score: {evaluation_results['avg_rouge1']:.4f}")
print(f"Average ROUGE-2 F-score: {evaluation_results['avg_rouge2']:.4f}")
print(f"Average ROUGE-L F-score: {evaluation_results['avg_rougeL']:.4f}")
print(f"Average BLEU Score: {evaluation_results['avg_bleu']:.4f}")
print("---------------------------------------")

# You can also inspect some of the generated answers vs ground truths
print("\n--- Sample of Generated vs. Ground Truth ---")
for i in range(min(3, evaluation_limit)): # Still showing max 3 samples for brevity
    print(f"Question: {evaluation_results['questions'][i]}")
    print(f"Ground Truth: {evaluation_results['ground_truths'][i]}")
    print(f"Generated Answer: {evaluation_results['generated_answers'][i]}\n")
print("--------------------------------------------")

Generating answers for 100 questions...
Processed 10/100 questions.
Processed 20/100 questions.
Processed 30/100 questions.
Processed 40/100 questions.
Processed 50/100 questions.
Processed 60/100 questions.
Processed 70/100 questions.
Processed 80/100 questions.
Processed 90/100 questions.
Processed 100/100 questions.
Calculating ROUGE and BLEU scores...

--- RAG Pipeline Evaluation Results ---
Evaluated on 100 samples.
Average ROUGE-1 F-score: 0.2464
Average ROUGE-2 F-score: 0.0668
Average ROUGE-L F-score: 0.1620
Average BLEU Score: 0.0241
---------------------------------------

--- Sample of Generated vs. Ground Truth ---
Question: Consider the Non-Disclosure Agreement between CopAcc and ToP Mentors; Does the document indicate that the Agreement does not grant the Receiving Party any rights to the Confidential Information?
Ground Truth: Any and all proprietary rights, including but not limited to rights to and in inventions, patent rights, utility models, copyrights, trademarks and

## **4. Conclusion** <font color=red> [5 marks] </font><br>

### **4.1 Conclusions and insights** <font color=red> [5 marks] </font><br>

#### **4.1.1** <font color=red> [5 marks] </font>
Conclude with the results here. Include the insights gained about the data, model pipeline, the RAG process and the results obtained.

### **Conclusions and Insights**

#### **Evaluation Summary:**

* **Quantitative Scores (100 samples):**
    * **ROUGE-1 F-score:** 0.2448 (Moderate word overlap)
    * **ROUGE-2 F-score:** 0.0659 (Low phrase overlap)
    * **ROUGE-L F-score:** 0.1613 (Moderate sentence structure overlap)
    * **BLEU Score:** 0.0240 (Very low exact match, typical for generative tasks)

#### **Insights Gained:**
* **Data Handling:** The pipeline successfully preprocesses legal documents, indicating effective initial data preparation.
* **RAG Process (Overall):** The system generally identifies correct "Yes/No" answers, suggesting effective retrieval of relevant context.
* **Generation Challenges:** The LLM often provides relevant but unformatted raw text snippets as answers, and sometimes includes less relevant details. This indicates that while retrieval is strong, the generation component (synthesis, summarization, and conciseness) needs improvement.
* **Model Performance:** The scores, particularly low ROUGE-2 and BLEU, suggest that the generated answers lack precise word and phrase similarity with human-written ground truths, pointing to a gap in the LLM's ability to rephrase or synthesize information accurately.

#### **Conclusion:**
The RAG pipeline effectively retrieves information from legal documents using Groq. However, refining the LLM's answer generation, focusing on conciseness and accurate synthesis of retrieved content, is crucial for improving overall performance and user experience. Enhancing pipeline robustness with retry mechanisms for API calls is also recommended.